### Suncal-Web Measurement Decision Risk Curve Calculator

---

In [1]:
%matplotlib ipympl
import ipywidgets as widgets

import numpy as np
import matplotlib.pyplot as plt

from suncal.risk import PFA_sweep_simple

In [2]:
xval = widgets.Dropdown(options=['ITP %', 'TUR', 'GBF', 'Bias %'], description='X Variable')
zval = widgets.Dropdown(options=['ITP %', 'TUR', 'GBF', 'Bias %'], value='TUR', description='Z Variable')
xrng = widgets.FloatRangeSlider(value=(55, 95), description='X Range', min=1, max=99, step=1)
zvals = widgets.Text(value='1.5, 2, 3, 4', description='Z Values')
itpconst = widgets.FloatText(value=95, description='ITP %', max=99, min=1)
turconst = widgets.FloatText(value=4, description='TUR')
gbtype = widgets.Dropdown(options=['Constant', 'RSS', 'Method 6'], description='Guardband')
gbfconst = widgets.FloatText(value=1, description='GB Constant', min=0)
biasconst = widgets.FloatText(value=0, description='Bias %', min=0)
yval = widgets.Dropdown(options=['PFA', 'PFR', 'Both'], description='Plot')
threed = widgets.Checkbox(value=False, description='3D')
replot = widgets.Button(description='Plot')

plt.ioff()
fig = plt.figure()
plt.ion()

def set_visibility():
    variables = [xval.value, zval.value]
    itpconst.layout.display = 'none' if 'ITP %' in variables else None
    turconst.layout.display = 'none' if 'TUR' in variables else None
    gbfconst.layout.display = 'none' if 'GBF' in variables else None
    biasconst.layout.display = 'none' if 'Bias %' in variables else None

def set_xval(change):
    xrng.value = {'TUR': (1.5, 4),
                  'ITP %': (55, 95),
                  'GBF': (0, 1),
                  'Bias %': (0, 100)}.get(xval.value)
    xrng.step = {'TUR': .1,
                  'ITP %': 1,
                  'GBF': .01,
                  'Bias %': 1}.get(xval.value)
    xrng.min = {'TUR': 0.1,
                'ITP %': 1,
                'GBF': 0,
                'Bias %': 0}.get(xval.value)
    xrng.max = {'TUR': 20,
                'ITP %': 99,
                'GBF': 2,
                'Bias %': 200}.get(xval.value)
    set_visibility()

def set_zval(change):
    zvals.value = {'TUR': '1.5, 2, 3, 4',
                   'GBF': '1, .95, .90, .85',
                   'ITP %': '75, 80, 85, 90, 95',
                   'Bias %': '0, 25, 50, 75'}.get(zval.value)
    set_visibility()

def on_plot(change):
    zvalues = np.array([float(x) for x in zvals.value.split(',')])
    xvalues = np.linspace(*xrng.value, num=20)

    xsteps = xvalues.copy()
    zsteps = zvalues.copy()
    if xval.value in ['ITP %', 'Bias %']:
        xsteps /= 100
    if zval.value in ['ITP %', 'Bias %']:
        zsteps /= 100

    guardband = {'Constant': gbfconst.value,
                 'RSS': 'rss',
                 'Method 6': 'dobbert'}.get(gbtype.value)
    fig.clf()

    yvars = [yval.value.lower()] if yval.value.lower() != 'both' else ['pfa', 'pfr']
    for k, yvar in enumerate(yvars):    
        pfx = PFA_sweep_simple(xvar=xval.value.strip(' %') if 'Bias' not in xval.value else 'pbias',
                           zvar=zval.value.strip(' %') if 'Bias' not in zval.value else 'pbias',
                           xvals=xsteps,
                           zvals=zsteps,
                           GBFdflt=guardband,
                           itpdflt=itpconst.value/100,
                           TURdflt=turconst.value,
                           pbias=biasconst.value/100,
                           risk=yvar)

        labels = {'tur': 'TUR', 'itp %': 'In-Tolerance Probability %',
                  'bias %': 'Process Distribution Bias', 'gbf': 'GBF'}
        xlabel = labels.get(xval.value.lower(), 'x')
        zlabel = labels.get(zval.value.lower(), 'z')
        ylabel = '{} %'.format(yvar.upper())

        if threed.value:
            ax = fig.add_subplot(1, len(yvars), k+1, projection='3d')
            xx, zz = np.meshgrid(xvalues, zvalues)
            ax.plot_surface(xx, zz, pfx*100, cmap='coolwarm')
            ax.set_zlabel(ylabel)
            ax.set_ylabel(zlabel)
        else:
            ax = fig.add_subplot(1, len(yvars), k+1)
            for i in range(len(zvalues)):
                ax.plot(xvalues, pfx[i, :]*100, label=str(zvalues[i]))
            ax.set_ylabel(ylabel)

        ax.set_xlabel(xlabel)
        if zval.value != 'none' and not threed.value:
            ax.legend(title=zlabel)

xval.observe(set_xval, names='value')
zval.observe(set_zval, names='value')
replot.on_click(on_plot)

set_visibility()
ctllayout = widgets.VBox([xval, zval, xrng, zvals, itpconst, turconst, gbtype, gbfconst, biasconst, yval, threed, replot])
layout = widgets.HBox([ctllayout, fig.canvas])

on_plot(None)

layout

---

#### Definitions

- **In-tolerance probability (ITP)**: The probability that any product (manufactured or calibrated) falls within the tolerance limits
- **Test Uncertainty Ratio (TUR)**: Ratio of tolerance to measurement uncertainty at 95% confidence
- **Guardband Factor (GBF)**: The guardbanded acceptance limit will be (tolerance × guardband factor). Guardband factor of 1 indicates no guardbanding is applied.
- **Bias**: Shift, as a percent of the tolerance, in the average value of product distribution.
- **Global False Accept Risk (PFA)**: probability that any product is falsely accepted by an inspection/test measurement
- **Global False Reject Risk (PFR)**: probability that any product is falsely rejected by an inspection/test measurement

#### Guardbanding Methods:

- **RSS**: Calculates guardband factor as $\sqrt{1-1/TUR^2}$
- **Method 6**: Calculates guardband factor as $1 - \frac{1.04 - \exp(0.38 \log(TUR) - 0.54)}{TUR}$ (See Dobbert, "A guardband strategy for managing false-accept risk")


---

#### Suncal

Suncal was developed by the Primary Standards Lab at Sandia National Laboratories. For more features and desktop version, see [https://sandiapsl.github.io](https://sandiapsl.github.io).

© 2019-2021 National Technology & Engineering Solutions of Sandia, LLC (NTESS).
Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government
retains certain rights in this software.